In [19]:
SSHEET_KEY = "11aR5WHloNn4N3Ns960reTCbxB_QUHeOU7RSqJzUyRbY"
SSHEET_WORKSHEET_ID = 1

In [20]:
import gspread

# creating client using our API key (requires either local file,
# or GitHub secret configuration)

gc = gspread.service_account(filename="service_account.json")

# retrieving spreadsheet

ssheet = gc.open_by_key(SSHEET_KEY)
if ssheet is None:
    raise Exception("Spreadsheet not found")

# printing spreadsheet info

print("Sheet title:", ssheet.title)
print("Worksheets:")
for i, wks in enumerate(ssheet.worksheets()):
    print(i, wks.title)

# retrieving worksheet

ws = ssheet.get_worksheet(SSHEET_WORKSHEET_ID)

Sheet title: Main List of platforms from Gleidson/Media Cloud/iOS+Android Apps
Worksheets:
0 Main
1 email_list_final
2 email_list
3 missing_ones_Owen
4 dictionary completed_restaurant_extraction
5 temp
6 Final
7 descriptive_statistics_scraping


In [26]:
# get all rows
rows = ws.get_all_values()

# separate header
header = rows[0]
rows = rows[1:]

# zip records
records = [dict(zip(header, row)) for row in rows]